In [53]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [42]:
import pandas as pd
import ujson
import json
pd.options.display.max_colwidth = 5000
import matplotlib
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
import random
import swifter
import collections

In [ ]:
data = pd.read_feather('../data/feather_files/data2019clean.feather')

# Step 2

In [76]:
#a
pol = data.query("tclass == 'political'").reset_index().text
nonpol = data.query("tclass == 'nonpolitical'").reset_index().text

In [77]:
tokens_p = word_tokenize(' '.join(pol))
tokens_np = word_tokenize(' '.join(nonpol)) 

In [84]:
print(f"Number of words are {len(tokens_p)} and {len(tokens_np)}")

counter_p = collections.Counter(tokens_p)
counter_np = collections.Counter(tokens_np)
print(f"Number of distinct words is {len(counter_p)} and {len(counter_np)}")
counter_p.most_common(3)
counter_np.most_common(3)


Number of words are 152212 and 2081118
Number of distinct words is 9386 and 31519


[('http', 8198), ('trump', 3934), ('house', 1528)]

[('get', 31903), ('like', 19674), ('say', 18876)]

In [115]:
worddict = pd.DataFrame({'word':counter_p.keys(),
                        'pol_occ': counter_p.values()})
worddict['nonpol_occ'] = worddict.word.swifter.apply(lambda x: counter_np.get(x,0))
temp = pd.DataFrame({'word':counter_np.keys(),
                'nonpol_occ': counter_np.values()})
temp['pol_occ'] = temp.word.swifter.apply(lambda x: counter_p.get(x,0))
worddict = pd.concat((worddict,temp)).drop_duplicates('word')\
.sort_values(by='pol_occ',ascending=False)
print(f"Number of distinct words is {len(worddict)}")
worddict = worddict.query('pol_occ >= 10 or nonpol_occ >= 10').reset_index(drop=True)
print(f"After removing low occurancies, we have {len(worddict)} wordds")

Pandas Apply:   0%|          | 0/9386 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/31519 [00:00<?, ?it/s]

Number of distinct words is 33600
After removing low occurancies, we have 8957 wordds


In [116]:
worddict['ssum'] = worddict.pol_occ + worddict.nonpol_occ
worddict['sc_n'] = worddict.pol_occ / worddict.ssum
worddict['sc_np'] = worddict.nonpol_occ / worddict.ssum

In [121]:
temp = worddict.sort_values(by='sc_n', ascending=False)[1:200]
worddict = worddict.sort_values(by='sc_np', ascending=False)[1:200]
worddict = pd.concat((temp, worddict))

In [126]:
worddict.drop_duplicates('word').sort_values(by='sc_n', ascending=False).reset_index()

,word,pol_occ,nonpol_occ,ssum,sc_n,sc_np
1638,caucus,14,0,14,1.0,0.0
1320,mccawley,18,0,18,1.0,0.0
1216,maddow,20,0,20,1.0,0.0
1441,gubernatorial,16,0,16,1.0,0.0
1217,coalition,20,0,20,1.0,0.0
...,...,...,...,...,...,...
6713,mph,0,17,17,0.0,1.0
6727,ahold,0,11,11,0.0,1.0
6726,oblige,0,93,93,0.0,1.0
6725,skinny,0,96,96,0.0,1.0


In [125]:
worddict

,word,pol_occ,nonpol_occ,ssum,sc_n,sc_np
1638,caucus,14,0,14,1.0,0.0
684,cspan,43,0,43,1.0,0.0
1968,reuters,10,0,10,1.0,0.0
697,nomination,42,0,42,1.0,0.0
146,judiciary,183,0,183,1.0,0.0
...,...,...,...,...,...,...
6942,altogether,0,23,23,0.0,1.0
6941,undies,0,66,66,0.0,1.0
6940,comfy,0,41,41,0.0,1.0
6939,boric,0,19,19,0.0,1.0
